Importação das bibliotecas

In [1]:
import pandas as pd
from mtcnn import MTCNN
import cv2

Definindo o dataframe como o conjunto de dados de treino

In [ ]:
df=pd.read_csv("../data/raw/fairface_label_train.csv")

Visualizando as raças presentes no conjunto de dados

In [ ]:
df.race.unique()

Nomeando cada dataframe somente com as imagens de uma única raça

In [ ]:
df_east_asian=df[df['race']=='East Asian']
df_indian=df[df['race']=='Indian']
df_black=df[df['race']=='Black']
df_white=df[df['race']=='White']
df_middle_eastern=df[df['race']=='Middle Eastern']
df_latino_hispanic=df[df['race']=='Latino_Hispanic']
df_southeast_asian=df[df['race']=='Southeast Asian']

Realizando o teste da aplicação do modelo MTCNN com uma única imagem. É possível visualizar vários atributos da imagem, porém só será utilizado um entre eles: confiança.

In [ ]:
img = cv2.cvtColor(cv2.imread("../data/processed/train/50.jpg"), cv2.COLOR_BGR2RGB)
detector = MTCNN()
detector.detect_faces(img)

Foi criada uma função para trazer o nível de confiança do rosto detectado que recebe como entrada o caminho da imagem

In [ ]:
def get_confidences(image_path):
    img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    detector = MTCNN()
    faces = detector.detect_faces(img)
    confidences = [face['confidence'] for face in faces]
    return confidences

Nesta célula foi atribuido uma raça por vez, visto que o algoritmo é muito pesado e demora mais de uma hora para processar as imagens.

In [ ]:
df_race=df_white

Foi criada uma nova coluna no dataframe chamada 'confidence'.

In [ ]:
df_race['confidence']=pd.NA

É possível visualizar o novo dataframe que será atualizado

In [ ]:
df_race

Esta iteração chama a função para aplicar o modelo MTCNN. Se houver mais de um rosto detectado, é apresenta uma mensagem no lugar do número de confiança. De forma, semelhante, também é apresentado se não foi possível detectar rostos ou houve algum erro para aplicar o modelo. Para todas as etnias branca e negra, foi processado todo o conjunto de dados de treino. Já para as demais etnias, apenas foi processada as 2000 primeiras imagens. O motivo deve-se a demora para processar cada imagem.

In [ ]:
# i=0
# for index, row in df_race.iterrows():  
#     if i>=14000: 
#         image_path_short = row['file']
#         image_path_long = '../data/processed/' + image_path_short
#         try:
#             list_conf=get_confidences(image_path_long)
#             if len(list_conf)==1:
#                 df_race.loc[index, 'confidence'] = float(list_conf[0])
#             elif len(list_conf)==0:
#                 df_race.loc[index, 'confidence'] = 'nenhum rosto detectado (=0)'
#             else:
#                 df_race.loc[index, 'confidence'] = 'mais de um rosto detectado'
#         except:
#             df_race.loc[index, 'confidence'] = 'não foi detectado nenhum rosto'
#     i+=1

Ao final são salvas em um arquivo .csv na pasta interim de dados.

In [ ]:
#df_race.to_csv('df_white_14000_to_end.csv')

## Avaliando a aplicação do MTCNN por etnia

Lendo cada resultado dos conjuntos para a raça negra

In [2]:
# Etnia negra:
df_black=pd.read_csv('../data/interim/df_black_to_2000.csv')
df_black_segundo_conjunto=pd.read_csv('../data/interim/df_black_2000_to_4000.csv')
df_black_terceiro_conjunto=pd.read_csv('../data/interim/df_black_4000_to_6000.csv')
df_black_quarto_conjunto=pd.read_csv('../data/interim/df_black_6000_to_8000.csv')
df_black_quinto_conjunto=pd.read_csv('../data/interim/df_black_8000_to_10000.csv')
df_black_sexto_conjunto=pd.read_csv('../data/interim/df_black_10000_to_end.csv')

Lendo cada resultado dos conjuntos para a raça branca

In [3]:
# Etnia branca:
df_white=pd.read_csv('../data/interim/df_white_to_2000.csv')
df_white_segundo_conjunto=pd.read_csv('../data/interim/df_white_2000_to_4000.csv')
df_white_terceiro_conjunto=pd.read_csv('../data/interim/df_white_4000_to_6000.csv')
df_white_quarto_conjunto=pd.read_csv('../data/interim/df_white_6000_to_8000.csv')
df_white_quinto_conjunto=pd.read_csv('../data/interim/df_white_8000_to_10000.csv')
df_white_sexto_conjunto=pd.read_csv('../data/interim/df_white_10000_to_12000.csv')
df_white_setimo_conjunto=pd.read_csv('../data/interim/df_white_12000_to_14000.csv')
df_white_oitavo_conjunto=pd.read_csv('../data/interim/df_white_14000_to_end.csv')


Lendo os arquivos das demais etnias

In [4]:
# Demais etnias:
df_east_asian=pd.read_csv('../data/interim/df_east_asian_to_2000.csv')
df_indian=pd.read_csv('../data/interim/df_indian_to_2000.csv')
df_latino_hispanic=pd.read_csv('../data/interim/df_latino_hispanic_to_2000.csv')
df_middle_eastern=pd.read_csv('../data/interim/df_middle_eastern_to_2000.csv')
df_southeast_asian=pd.read_csv('../data/interim/df_southeast_asian_to_2000.csv')

Definindo as funções que serão utilizadas

Função para calcular a média das confianças:

In [5]:
def calcula_media(df):
    # Converte a coluna para numérico, forçando erros a NaN
    df['confidence_only_numeric'] = pd.to_numeric(df['confidence'], errors='coerce')
    # Remove os valores NaN
    df_cleaned = df.dropna(subset=['confidence_only_numeric'])
    # Calcula a média dos valores restantes
    mean_value = df_cleaned['confidence_only_numeric'].mean()
    return mean_value

Função para realizar o print da quantidade de imagens que não foi possível detectar rostos:

In [6]:
def print_nao_detectado(df):
    cont_nao_detec=df[df['confidence']=='nenhum rosto detectado (=0)']['file'].count()
    print('Não foram detectados rostos em ', cont_nao_detec, 'imagens')

Função para realizar o print da quantidade de imagens que foram detectados mais de um rosto:

In [7]:
def print_mais_rostos(df):
    cont_rostos=df[df['confidence']=='mais de um rosto detectado']['file'].count()
    print('Foram detectados mais de um rosto em', cont_rostos, 'imagens')

Função para realizar o print da quantidade de imagens que houve problema na aplicação do modelo MTCNN:

In [8]:
def print_erro_modelo(df):
    cont_problema_mod=df[df['confidence']=='não foi detectado nenhum rosto']['file'].count()
    print('Houve problema na detecção em', cont_problema_mod, 'imagens')

### Investigação dos erros por etnias

Inicialmente serão analisadas apenas 2000 imagens por etnia.

Para a etnia negra:

In [15]:
print('A média de confiança na detecção foi de: ', round(calcula_media(df_black),4))
print_nao_detectado(df_black)
print_erro_modelo(df_black)
print_mais_rostos(df_black)

A média de confiança na detecção foi de:  nan
Não foram detectados rostos em  0 imagens
Houve problema na detecção em 0 imagens
Foram detectados mais de um rosto em 15 imagens


Para a etnia do leste asiático:

In [16]:
print('A média de confiança na detecção foi de: ', round(calcula_media(df_east_asian),4))
print_nao_detectado(df_east_asian)
print_erro_modelo(df_east_asian)
print_mais_rostos(df_east_asian)

A média de confiança na detecção foi de:  0.993
Não foram detectados rostos em  0 imagens
Houve problema na detecção em 0 imagens
Foram detectados mais de um rosto em 28 imagens


Para a etnia indiana:

In [17]:
print('A média de confiança na detecção foi de: ', round(calcula_media(df_indian),4))
print_nao_detectado(df_indian)
print_erro_modelo(df_indian)
print_mais_rostos(df_indian)

A média de confiança na detecção foi de:  nan
Não foram detectados rostos em  15 imagens
Houve problema na detecção em 0 imagens
Foram detectados mais de um rosto em 2 imagens


Para a etnia latino hispânico:

In [18]:
print('A média de confiança na detecção foi de: ', round(calcula_media(df_latino_hispanic),4))
print_nao_detectado(df_latino_hispanic)
print_erro_modelo(df_latino_hispanic)
print_mais_rostos(df_latino_hispanic)

A média de confiança na detecção foi de:  nan
Não foram detectados rostos em  10 imagens
Houve problema na detecção em 325 imagens
Foram detectados mais de um rosto em 3 imagens


Para a etnia do oriente médio:

In [19]:
print('A média de confiança na detecção foi de: ', round(calcula_media(df_middle_eastern),4))
print_nao_detectado(df_middle_eastern)
print_erro_modelo(df_middle_eastern)
print_mais_rostos(df_middle_eastern)

A média de confiança na detecção foi de:  nan
Não foram detectados rostos em  16 imagens
Houve problema na detecção em 181 imagens
Foram detectados mais de um rosto em 1 imagens


Para a etnia do sudeste asiático:

In [20]:
print('A média de confiança na detecção foi de: ', round(calcula_media(df_southeast_asian),4))
print_nao_detectado(df_southeast_asian)
print_erro_modelo(df_southeast_asian)
print_mais_rostos(df_southeast_asian)

A média de confiança na detecção foi de:  0.9945
Não foram detectados rostos em  95 imagens
Houve problema na detecção em 0 imagens
Foram detectados mais de um rosto em 18 imagens


Para a etnia branca:

In [21]:
print('A média de confiança na detecção foi de: ', round(calcula_media(df_white),4))
print_nao_detectado(df_white)
print_erro_modelo(df_white)
print_mais_rostos(df_white)

A média de confiança na detecção foi de:  0.9907
Não foram detectados rostos em  0 imagens
Houve problema na detecção em 0 imagens
Foram detectados mais de um rosto em 36 imagens


#### Análise de dois conjuntos de dados completos

Combinando as colunas dos dataframes da etnia negra:

In [22]:
col_completa_confidence_black = df_black['confidence'].combine_first(df_black_segundo_conjunto['confidence']) \
                                        .combine_first(df_black_terceiro_conjunto['confidence']) \
                                        .combine_first(df_black_quarto_conjunto['confidence']) \
                                        .combine_first(df_black_quinto_conjunto['confidence']) \
                                        .combine_first(df_black_sexto_conjunto['confidence'])

In [23]:
df_black['confidence']=col_completa_confidence_black

Análise da base de dados completa da etnia negra:

In [24]:
print('A média de confiança na detecção foi de: ', round(calcula_media(df_black),4))
print_nao_detectado(df_black)
print_erro_modelo(df_black)
print_mais_rostos(df_black)

A média de confiança na detecção foi de:  0.9926
Não foram detectados rostos em  537 imagens
Houve problema na detecção em 0 imagens
Foram detectados mais de um rosto em 82 imagens


Combinando as colunas dos dataframes da etnia branca:

In [25]:
col_completa_confidence_white = df_white['confidence'].combine_first(df_white_segundo_conjunto['confidence']) \
                                        .combine_first(df_white_terceiro_conjunto['confidence']) \
                                        .combine_first(df_white_quarto_conjunto['confidence']) \
                                        .combine_first(df_white_quinto_conjunto['confidence']) \
                                        .combine_first(df_white_sexto_conjunto['confidence']) \
                                        .combine_first(df_white_setimo_conjunto['confidence']) \
                                        .combine_first(df_white_oitavo_conjunto['confidence'])

In [26]:
df_white['confidence']=col_completa_confidence_white

In [ ]:
print('A média de confiança na detecção foi de: ', round(calcula_media(df_white),4))
print_nao_detectado(df_white)
print_erro_modelo(df_white)
print_mais_rostos(df_white)

Como é possível observar, apesar do conjunto de dados da etnia branca ser maior, a quantidade de erros na detecção de rostos foi maior para esta etnia.

### Avaliando os resultados do modelo MTCNN por gênero e raça

In [5]:
list_dfs=[df_white, df_black, df_indian, df_latino_hispanic, df_middle_eastern, df_southeast_asian]


Avaliando os erros por etnia agrupando por gênero

In [14]:
for df in list_dfs:
    print(df['race'][0])
    df_female=df[df['gender']=='Female']
    df_male=df[df['gender']=='Male']
    print("Rostos femininos")
    print_nao_detectado(df_female)
    print("Rostos masculinos")
    print_nao_detectado(df_male)

White
Rostos femininos
Não foram detectados rostos em  0 imagens
Rostos masculinos
Não foram detectados rostos em  0 imagens
Black
Rostos femininos
Não foram detectados rostos em  0 imagens
Rostos masculinos
Não foram detectados rostos em  0 imagens
Indian
Rostos femininos
Não foram detectados rostos em  8 imagens
Rostos masculinos
Não foram detectados rostos em  7 imagens
Latino_Hispanic
Rostos femininos
Não foram detectados rostos em  4 imagens
Rostos masculinos
Não foram detectados rostos em  6 imagens
Middle Eastern
Rostos femininos
Não foram detectados rostos em  5 imagens
Rostos masculinos
Não foram detectados rostos em  11 imagens
Southeast Asian
Rostos femininos
Não foram detectados rostos em  35 imagens
Rostos masculinos
Não foram detectados rostos em  60 imagens


Avaliando os erros da raça branca por cada gênero

In [27]:
print('Female')
print_nao_detectado(df_white[df_white['gender']=='Female'])
print('Male')
print_nao_detectado(df_white[df_white['gender']=='Male'])

Female
Não foram detectados rostos em  486 imagens
Male
Não foram detectados rostos em  606 imagens


Avaliando os erros da raça negra por cada gênero

In [28]:
print('Female')
print_nao_detectado(df_black[df_white['gender']=='Female'])
print('Male')
print_nao_detectado(df_black[df_white['gender']=='Male'])

Female
Não foram detectados rostos em  236 imagens
Male
Não foram detectados rostos em  301 imagens


C:\Users\DELL\AppData\Local\Temp\ipykernel_19064\3069438064.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print_nao_detectado(df_black[df_white['gender']=='Female'])
C:\Users\DELL\AppData\Local\Temp\ipykernel_19064\3069438064.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print_nao_detectado(df_black[df_white['gender']=='Male'])


### Avaliando a existência de viés no modelo

Todas os diferentes dataframes foram unidos novamente em um só

In [6]:
# Concatenate all DataFrames
combined_df = pd.concat(list_dfs)

# Sort the combined DataFrame by the index column
df_mtcnn = combined_df.sort_values(by='Unnamed: 0')

# Reset the index if you want a clean integer index
df_mtcnn.reset_index(drop=True, inplace=True)

Foram retirados os valores NaN do conjunto dos arquivos

In [7]:
df_mtcnn = df_mtcnn[df_mtcnn['file'].notna()]

Foram retirados onde não foi aplicado o modelo

In [8]:
df_mtcnn = df_mtcnn[df_mtcnn['confidence'].notna()]

Foi aplicado uma nova coluna em que o valor torna-se falso para as imagens em que não foi possível detectar o rosto

In [9]:
df_mtcnn['face_detected'] = df_mtcnn['confidence'] != 'nenhum rosto detectado (=0)'

O seguinte código calcula as métricas para avaliação de viés

In [14]:
from fairlearn.metrics import MetricFrame, true_positive_rate, true_negative_rate, false_positive_rate, false_negative_rate, selection_rate


# Verificar se a coluna 'service_test' representa a verdade sobre a presença de um rosto
df['face_present'] = df['service_test']  # Supondo que 'service_test' é True se um rosto está presente
# Definir as colunas para avaliação
y_true = df['face_present']
y_pred = df['face_detected']
sensitive_features = df[['race']]

# Calcular métricas usando Fairlearn
metrics = {
    'true_positive_rate': true_positive_rate,
    'false_positive_rate': false_positive_rate,
    'true_negative_rate': true_negative_rate,
    'false_negative_rate': false_negative_rate,
    'selection_rate': selection_rate
}

metric_frame = MetricFrame(metrics=metrics, y_true=y_true, y_pred=y_pred, sensitive_features=sensitive_features)

# Paridade Demográfica: selection_rate
demographic_parity = metric_frame.by_group['selection_rate']
print("\nParidade Demográfica (Taxa de Seleção por Raça):")
print(demographic_parity)

# Igualdade de Oportunidade: true_positive_rate
equalized_opportunity = metric_frame.by_group['true_positive_rate']
print("\nIgualdade de Oportunidade (Taxa de Verdadeiros Positivos por Raça):")
print(equalized_opportunity)

# Igualdade de Chance: true_positive_rate e false_positive_rate
true_positive_rates = metric_frame.by_group['true_positive_rate']
false_positive_rates = metric_frame.by_group['false_positive_rate']

# Calculate the maximum and minimum difference for TPR and FPR by each sensitive group
max_diff_tpr = true_positive_rates.groupby(level='race').transform(lambda x: x.max() - x.min())
max_diff_fpr = false_positive_rates.groupby(level='race').transform(lambda x: x.max() - x.min())

equalized_chance = pd.DataFrame({
    'true_positive_rate': true_positive_rates,
    'false_positive_rate': false_positive_rates,
    'max_diff_tpr': max_diff_tpr,
    'max_diff_fpr': max_diff_fpr
})

equalized_chance['equalized_chance'] = equalized_chance['max_diff_tpr'] + equalized_chance['max_diff_fpr']

print("\nIgualdade de Chance (Taxas de Verdadeiros Positivos e Falsos Positivos por Raça):")
print(equalized_chance['equalized_chance'])

# Compare true positive rate and false positive rate for each group
predictive_parity = (true_positive_rates - false_positive_rates).abs()

print("Predictive Parity (Absolute Difference between True Positive Rate and False Positive Rate by Race):")
print(predictive_parity)

# Compute false positive rate and false negative rate by group
false_positive_rates = metric_frame.by_group['false_positive_rate']
false_negative_rates = metric_frame.by_group['false_negative_rate']

# Compare false positive rate and false negative rate for each group
treatment_equality = (false_positive_rates - false_negative_rates).abs()

print("Equality of Treatment (Absolute Difference between False Positive Rate and False Negative Rate by Race):")
print(treatment_equality)


Paridade Demográfica (Taxa de Seleção por Raça):
race
Black              1.000000
Indian             0.946996
Latino_Hispanic    0.970414
Middle Eastern     0.919192
Southeast Asian    0.952547
White              1.000000
Name: selection_rate, dtype: float64

Igualdade de Oportunidade (Taxa de Verdadeiros Positivos por Raça):
race
Black              1.000000
Indian             0.942446
Latino_Hispanic    0.978261
Middle Eastern     0.914062
Southeast Asian    0.950820
White              1.000000
Name: true_positive_rate, dtype: float64

Igualdade de Chance (Taxas de Verdadeiros Positivos e Falsos Positivos por Raça):
race
Black              0.0
Indian             0.0
Latino_Hispanic    0.0
Middle Eastern     0.0
Southeast Asian    0.0
White              0.0
Name: equalized_chance, dtype: float64
Predictive Parity (Absolute Difference between True Positive Rate and False Positive Rate by Race):
race
Black              0.000000
Indian             0.008943
Latino_Hispanic    0.013261
Mid